In [92]:
from keras.layers.core import Activation, Dense, Dropout
from keras.layers import concatenate,Reshape,Add,LSTM,Multiply
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential,Model
from keras import Model
from keras import Input

import cython
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from time import time
from gensim.models import Word2Vec
from keras.initializers import Constant
from keras.utils import to_categorical
from keras.preprocessing.text import text_to_word_sequence,Tokenizer
from keras.preprocessing.sequence import pad_sequences,TimeseriesGenerator

from sklearn.model_selection import train_test_split


## Define parameters

In [ ]:
seqLen=100
dateShape=(10,)
signShape=(10,)
textShape=(10,10)

## Define structure

### Date input

In [ ]:
inputDate=Input(shape=dateShape)
denseD1=Dense(8,activation="relu")(inputDate)
outputD=Dense(16,activation="relu")(denseD1)

### Sign input

In [ ]:
inputSign=Input(shape=signShape)
denseS1=Dense(8,activation="relu")(inputSign)
outputS=Dense(16,activation="relu")(denseS1)

### Text input

In [ ]:
inputText=Input(shape=textShape)

### Concatenate date and sign

In [ ]:
combined=concatenate([outputF,outputS])

## input->add tanh->lstm->mul sigm->lstm

### Define tanh dense

In [ ]:
denseTanh=Dense(100,activation='tanh')(combined)
reshapedTanh=Reshape(textShape)(denseTanh)

### Define sigmoid dense

In [ ]:
denseSigmoid=Dense(100,activation='sigmoid')(combined)
reshapedSigmoid=Reshape(textShape)(denseSigmoid)

### Add 

In [ ]:
add1=Add()([reshapedTanh,inputText])
lstm1=LSTM(100)(add1)
reshapedLstm=Reshape(textShape)(lstm1)
mul1=Multiply()([reshapedLstm,reshapedSigmoid])
lstm2=LSTM(100)(mul1)

## Define model

In [ ]:
#model=Model(inputs=[inputDate,inputSign,inputText],outputs=[lstm2])

### Загрузим данные

размер словаря - 84267 слов,количество предложений - 73477, максимальная длинна текста - 398 символов, максимальное количество слов в предложении - 190


In [93]:
data=np.array(pd.read_csv('data.csv'))

### Проведем токенизацию

То есть разобьем исходные предложения на слова

In [94]:
def processText(data):
    tokens=[]
    for line in data:
        newToken=text_to_word_sequence(text=line[2],filters='!"#$%&amp;()*+,-./:;&lt;=>?@[\\]^_`{|}~\t\n\ufeff',
                                  lower=True,split=' ')
        tokens.append(newToken)
    return tokens

In [95]:
wordLists=processText(data)
print(wordLists[0])

['любые', 'разногласия', 'во', 'мнениях', 'скоро', 'улягутся', 'а', 'вы', 'продолжайте', 'делать', 'как', 'делали', 'но', 'постарайтесь', 'не', 'наступать', 'на', 'ноги', 'слишком', 'многим', 'иначе', 'ваши', 'сегодняшние', 'действия', 'сыграют', 'против', 'вас', 'в', 'будущем']


### Заменим слова в предложениях на соответсвующие в словаре индексы

In [96]:
num_words = 84267
tokenizer = Tokenizer(
    num_words=num_words,
    filters='!"#$%&amp;()*+,-—./:;&lt;=>?@[\\]^_`{|}~\t\n\xa0\ufeff',
    lower=True,
    split=' ',
    char_level=False) 

tokenizer.fit_on_texts(wordLists) 
sequences = np.array(tokenizer.texts_to_sequences(wordLists))
print(sequences[0])

[293, 448, 66, 8385, 4715, 15933, 22, 8, 10529, 336, 34, 10057, 25, 127, 3, 16752, 7, 12640, 169, 755, 254, 43, 3549, 279, 4619, 953, 12, 2, 614]


### Зафиксируем характеристики выборки

In [97]:
sentLen = len(max(sequences, key = len)) # max_len
word2index = tokenizer.word_index       #word_index
wordsNum = len(tokenizer.word_index) + 1 # num_words
embeddingDim=300

### Дополним предожения нулями до одной длины

In [98]:
sequences = pad_sequences(sequences = sequences, maxlen = sentLen,padding='post')
print(sequences[0])
print(sequences[0].shape)
print(sequences.shape)

[  293   448    66  8385  4715 15933    22     8 10529   336    34 10057
    25   127     3 16752     7 12640   169   755   254    43  3549   279
  4619   953    12     2   614     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [99]:
w2v = Word2Vec.load("word2vec.model")
embeddingMatrix = np.zeros((wordsNum, embeddingDim))
for word, i in tokenizer.word_index.items():
    if i > wordsNum: #если индекс превышает кол-во слов в словаре, то скипаем  
        continue
    embeddingVector = w2v[word] #получаем вектор соответствущий слову в модели word2vec
    if embeddingVector is not None:  #если слово отсутствует в словаре word2vec, то оно в матрице np.zeroes останется равным 0
        embeddingMatrix[i] = embeddingVector #если слово найдено в словаре токенизатора, то в embedding_matrix проставляем вектор соответствующий слову

C:\Users\Администратор\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [100]:
std=embeddingMatrix.std(axis=0)
mean=embeddingMatrix.mean(axis=0)
for i in range(embeddingMatrix.shape[0]):
    for j in range(embeddingMatrix.shape[1]):
        embeddingMatrix[i][j]=(embeddingMatrix[i][j]-mean[j])/std[j]

In [101]:
embeddingLayer = Embedding(
    input_dim = wordsNum, 
    output_dim = embeddingDim, 
    embeddings_initializer = Constant(embeddingMatrix),
    input_length = sentLen, 
    trainable = False)    

### Приведем датасет к специальному виду

 Например для текста "The sky was falling due to apocalypse " будет
 
 "The sky" -> was
 
 "sky was" -> falling
 
 " was falling " -> due 
 
 " falling due" -> to
 
 "due to " -> apocalypse
 


In [102]:
def word2idx(word):
    return w2v.wv.vocab[word].index
def idx2word(idx):
    return w2v.wv.index2word[idx]

In [103]:
seqlen=10
step=1
inputSent = []
labelSent = []
for i,line in enumerate(sequences):
    for j in range(0, len(line) - seqlen, step):
        inputSent.append(line[j:j + seqlen])
        labelSent.append(line[j + seqlen])
        

In [111]:
X=np.array(inputSent)
Y=np.zeros((X.shape[0],embeddingDim),dtype='float32')

for i,index in enumerate(labelSent):
    Y[i]=w2v[idx2word(index)]

C:\Users\Администратор\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [112]:
Y.shape

(13225860, 300)

In [113]:
Y[0][:10]

array([-0.7566195 , -0.16491489, -0.38569096,  0.16882984,  0.8041008 ,
       -0.16769701, -0.9228554 , -2.150708  ,  0.28348798,  1.2643976 ],
      dtype=float32)

In [67]:
del embeddingMatrix

In [48]:
np.save('Y.npy',Y)

In [54]:
Y=np.load('Y.npy')

In [114]:
for j in range(Y.shape[1]):
    Y[:,j]=(Y[:,j]-mean[j])/std[j]
    if j%1000==0:
        print(j)


0


KeyboardInterrupt: 

In [24]:
x_train_, x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2) #, random_state = 42)

In [110]:
del X
del Y

In [69]:
del Y

In [28]:

print("Размер embedding матрицы:", num_words, "x", embeddingDim)
embeddingMatrix = np.zeros((num_words+1, embeddingDim))
for word, i in tokenizer.word_index.items():
    if i > num_words: #если индекс превышает кол-во слов в словаре, то скипаем  
        continue
    embeddingVector = w2v[word] #получаем вектор соответствущий слову в модели word2vec
    if embeddingVector is not None:  #если слово отсутствует в словаре word2vec, то оно в матрице np.zeroes останется равным 0
        embeddingMatrix[i] = embeddingVector #если слово найдено в словаре токенизатора, то в embedding_matrix проставляем вектор соответствующий слову



Размер embedding матрицы: 84267 x 300


C:\Users\Администратор\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [30]:
embeddingLayer = Embedding(input_dim = num_words, 
                              output_dim = embeddingDim, 
                              embeddings_initializer = Constant(embeddingMatrix),
                              input_length =sentLen, 
                              trainable = False)    

In [ ]:
modelGRU = Sequential()
#embedding_layer = w2v.wv.get_keras_embedding(train_embeddings=False)
modelGRU.add(embeddingLayer)
#modelGRU.add(Embedding(num_words, embedding_size))
modelGRU.add(SpatialDropout1D(0.2))
modelGRU.add(Bidirectional(GRU(40, return_sequences=True)))
modelGRU.add(Bidirectional(GRU(40)))
#modelGRU.add(LSTM(8,return_sequences=True ))
#modelGRU.add(LSTM(8))
modelGRU.add(Dropout(0.2))
modelGRU.add(Dense(64,activation = 'relu'))
modelGRU.add(Dropout(0.2))
modelGRU.add(Dense(num_classes,activation = 'sigmoid'))
modelGRU.summary()


In [ ]:
modelGRU = buildModel(embedding_layer)
#modelGRU.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=Adam(lr=1e-4))
modelGRU.compile(loss='binary_crossentropy', metrics=[AUC(name='auc')], optimizer=Adam(lr=1e-4))
historyGRU = modelGRU.fit(X_train, Y_train, batch_size=64, epochs=20, validation_data=(X_test, Y_test))